In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("Data/gender_age_train.csv")
test = pd.read_csv("Data/gender_age_test.csv")


In [3]:
def tohex_str(val, nbits=64):
    hex_value = hex((val + (1 << nbits)) % (1 << nbits))
    str_hex = str(hex_value)[2:-1]
    str_hex = str_hex.zfill(16)
    return str_hex

def tohex_int(val, nbits=64):
    hex_value = hex((val + (1 << nbits)) % (1 << nbits))
    #str_hex = str(hex_value)[2:]
    #str_hex = str_hex[:-1]
    #return str_hex
    return hex_value

def just_int(val):
    s = [int(x) for x in ("%.20d" % abs(val))]
    if val < 0:
        s[0] = 1
    return pd.Series(s)

    

In [4]:
def row_intechange(df):
    wihtout_hex = df
    wihtout_hex['row_number'] = np.arange(df.shape[0])
    wihtout_hex['row_number'] = wihtout_hex['row_number'] / df.shape[0]
    wihtout_hex = df.sort('device_id')
    wihtout_hex['without_hex_row'] = np.arange(df.shape[0])
    wihtout_hex['without_hex_row'] = wihtout_hex['without_hex_row'] / df.shape[0]
    wihtout_hex['hex'] = wihtout_hex['device_id'].apply(tohex_str)
    wihtout_hex = wihtout_hex.sort('hex')
    wihtout_hex['hex_row'] = np.arange(wihtout_hex.shape[0])
    wihtout_hex['hex_row']  = wihtout_hex['hex_row'] / df.shape[0]
    return wihtout_hex.sort('row_number')

In [5]:
train['train_index'] = np.arange(train.shape[0])
train['train_index'] = train['train_index'] / train.shape[0]
train['original_index'] = train['train_index']
test['test_index'] = np.arange(test.shape[0])
test['test_index'] = test['test_index'] / test.shape[0]
test['original_index'] = test['test_index']

In [40]:
train_test = pd.concat([train, test], axis=0).reset_index()

In [41]:
sorted_train_test = train_test.sort(['index'])
sample = sorted_train_test
sample.head(15)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,index,age,device_id,gender,group,original_index,test_index,train_index
0,0,35,-8076087639492063270,M,M32-38,0.000000,NaN,0.000000
74645,0,NaN,1002079943728939269,NaN,NaN,0.000000,0.000000,NaN
74646,1,NaN,-1547860181818787117,NaN,NaN,0.000009,0.000009,NaN
1,1,35,-2897161552818060146,M,M32-38,0.000013,NaN,0.000013
2,2,35,-8260683887967679142,M,M32-38,0.000027,NaN,0.000027
74647,2,NaN,7374582448058474277,NaN,NaN,0.000018,0.000018,NaN
3,3,30,-4938849341048082022,M,M29-31,0.000040,NaN,0.000040
74648,3,NaN,-6220210354783429585,NaN,NaN,0.000027,0.000027,NaN
74649,4,NaN,-5893464122623104785,NaN,NaN,0.000036,0.000036,NaN
4,4,30,245133531816851882,M,M29-31,0.000054,NaN,0.000054


In [42]:
real_index = []

small_set = []

for (i, row) in sample.iterrows():
    if np.isnan(row['age']) == False:
        if len(small_set) > 0:
            j = 0
            while(j < len(small_set)):
                if small_set[j][1] < row['device_id']:
                    real_index.append((small_set[j][0],small_set[j][1],small_set[j][2],
                                       small_set[j][3],small_set[j][4], small_set[j][5]))
                else:
                    break
                j+= 1
            small_set = small_set[j:]
        #print 'add device: ', row['device_id']
        real_index.append((row['age'],row['device_id'],row['gender'],row['group'],row['test_index'],row['train_index']))
    else:
        #print 'wait for confirm device: ', row['device_id']
        small_set.append((row['age'],row['device_id'],row['gender'],row['group'],row['test_index'],row['train_index']))
    #print i, ': ' , len(real_index), len(small_set)


real_index = real_index + small_set
#print real_index
result = pd.DataFrame(real_index)

In [43]:
result['sort_index'] = np.arange(result.shape[0])

In [44]:
result['sort_index'] = result['sort_index'] / result.shape[0]

In [45]:
result.columns = ['age','device_id','gender','group','test_index','train_index','sort_index']

In [8]:
phone_data = pd.read_csv("Data/phone_brand_device_model.csv").drop_duplicates('device_id',keep='first')

In [47]:
train_test = result.merge(phone_data, how='left', on='device_id')

In [48]:
phone_encoder = LabelEncoder()
train_test['phone_brand'] = phone_encoder.fit_transform(train_test.phone_brand)
model_encoder = LabelEncoder()
train_test['device_model'] = model_encoder.fit_transform(train_test.device_model)

In [49]:
train_test.head()

,age,device_id,gender,group,test_index,train_index,sort_index,phone_brand,device_model
0,35,-8076087639492063270,M,M32-38,NaN,0.000000,0.000000,51,749
1,35,-2897161552818060146,M,M32-38,NaN,0.000013,0.000005,51,749
2,35,-8260683887967679142,M,M32-38,NaN,0.000027,0.000011,51,749
3,30,-4938849341048082022,M,M29-31,NaN,0.000040,0.000016,51,1524
4,30,245133531816851882,M,M29-31,NaN,0.000054,0.000021,51,753


In [ ]:
from sklearn.cluster import MiniBatchKMeans

clf = MiniBatchKMeans(n_clusters=60000, batch_size=1000, verbose=0)
us_result = clf.fit_predict(train_test[['sort_index','phone_brand','device_model']])

In [38]:
us_result

array([ 0.25378508,  0.25378508,  0.25378508, ...,  0.14688294,
        0.14688294,  0.23948479])

In [34]:

l = np.max(us_result)
us_result = us_result / 49999.
train_test['kmean_50000'] = us_result

d = train_test.dropna(subset=['train_index'])
np.save('WorkSpace/kmean_50000_train', d.kmean_50000.as_matrix())

d = train_test.dropna(subset=['test_index'])
np.save('WorkSpace/kmean_50000_test', d.kmean_50000.as_matrix())

In [23]:
np.save('WorkSpace/kmean_10000', us_result)

In [24]:
del us_result
del clf

In [ ]:
d = result.dropna(subset=['test_index'])
np.save('WorkSpace/both_row_test',d['both_row_number'].as_matrix())

In [22]:
tx = train.reset_index().sort(['index','device_id'])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [5]:
train = row_intechange(train)
test = row_intechange(test)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [11]:
train.columns = [u'device_id', u'gender', u'age', u'group', u'train_row_number',
       u'train_without_hex_row', u'hex', u'train_hex_row']

In [7]:
train.columns = [u'device_id', u'test_row_number', u'test_without_hex_row', u'hex', u'test_hex_row']
test.columns = [u'device_id', u'test_row_number', u'test_without_hex_row', u'hex', u'test_hex_row']

In [37]:
train_test = pd.concat([train,test], axis=0)

In [38]:
train_test = train_test.reset_index()
train_test.tail()

,index,age,device_id,gender,group,hex,test_hex_row,test_row_number,test_without_hex_row,train_hex_row,train_row_number,train_without_hex_row
186711,112066,NaN,4280900819321920929,NaN,NaN,3b68d09d2e5c9da1,0.232228,0.999955,0.733847,NaN,NaN,NaN
186712,112067,NaN,818534825520551359,NaN,NaN,0b5c0516477055bf,0.043803,0.999964,0.545422,NaN,NaN,NaN
186713,112068,NaN,-8956851351560395765,NaN,NaN,83b2df26b443e40b,0.513112,0.999973,0.014732,NaN,NaN,NaN
186714,112069,NaN,6097318236795836256,NaN,NaN,549e06a1a2ba1760,0.329425,0.999982,0.831045,NaN,NaN,NaN
186715,112070,NaN,622421180514002079,NaN,NaN,08a348c41537649f,0.033274,0.999991,0.534893,NaN,NaN,NaN


In [39]:
sort_hex_train_test = train_test.sort(['index','hex'])
sort_hex_train_test['both_row_number'] = np.arange(sort_hex_train_test.shape[0])
sort_hex_train_test['both_row_number'] = sort_hex_train_test['both_row_number']/ sort_hex_train_test.shape[0]
sort_hex_train_test = sort_hex_train_test.sort()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [53]:
d = sort_hex_train_test.dropna(subset=['train_hex_row'])

In [54]:
np.save('WorkSpace/both_row_train',d['both_row_number'].as_matrix())

In [55]:
d = sort_hex_train_test.dropna(subset=['test_hex_row'])
np.save('WorkSpace/both_row_test',d['both_row_number'].as_matrix())

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


In [8]:
r_train = train[['row_number','without_hex_row','hex_row']]
r_test = test[['row_number','without_hex_row','hex_row']]

In [11]:
r_test.shape

(112071, 3)

In [12]:
np.save('WorkSpace/r_train', r_train.as_matrix())
np.save('WorkSpace/r_test', r_test.as_matrix())

In [4]:
events = pd.read_csv("Data/events.csv")
appevents = pd.read_csv("Data/app_events.csv")

In [57]:
train.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [59]:
events.head()

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [63]:
only_train_events = events.merge(train, how='left', on='device_id', suffixes=('_l','_r')).dropna()

In [8]:
events_count = events.groupby('device_id').count()['event_id'].reset_index()

In [11]:
events_count['device_hex'] = events_count['device_id'].apply(tohex_str)

In [13]:
def extract_first_four(val):
    ff = val[0:4]
    mf = val[4:8]
    nf = val[8:12]
    bf = val[12:]
    return pd.Series([ff, mf, nf, bf])

In [23]:
extracted = events_count['device_hex'].apply(extract_first_four)

In [24]:
events_count = events_count.join(extracted, how='left')

In [26]:
events_count.columns = [u'device_id', u'event_id', u'device_hex', u'af', u'bf', u'cf', u'df']

In [41]:
events['timestamp'] = pd.to_datetime(events['timestamp'])

In [71]:
events['date_part'] = events['timestamp'].apply(lambda x: str(x.month) + '-' + str(x.day))

In [72]:
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder

def save_sparse(filename, xmtr):
    np.savez(filename,data = xmtr.data ,indices= xmtr.indices,
             indptr =xmtr.indptr, shape=xmtr.shape )
    
def load_sparse(filename):
    tmp = np.load(filename)
    return csr_matrix((tmp['data'], tmp['indices'], tmp['indptr']), shape= tmp['shape'])

In [73]:
date_encoder = LabelEncoder().fit(events.date_part)
events['date_part'] = date_encoder.transform(events.date_part)
date_len_ = len(date_encoder.classes_)
print 'total number of target classes: ', date_len_, date_encoder.classes_

total number of target classes:  9 ['4-30' '5-1' '5-2' '5-3' '5-4' '5-5' '5-6' '5-7' '5-8']


In [75]:
dx = events.merge(train, how='left', on='device_id', suffixes=('_l','_r')).dropna()

In [76]:
dx.sort('device_id')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,event_id,device_id,timestamp,longitude,latitude,date_part,gender,age,group
2425796,2425797,-9222956879900151005,2016-05-07 11:54:59,113.24,23.19,7,M,36,M32-38
379328,379329,-9222956879900151005,2016-05-06 15:40:43,113.24,23.19,6,M,36,M32-38
2213932,2213933,-9222956879900151005,2016-05-06 15:39:44,113.24,23.19,6,M,36,M32-38
519593,519594,-9222956879900151005,2016-05-06 15:35:28,113.24,23.19,6,M,36,M32-38
514595,514596,-9222956879900151005,2016-05-07 12:17:29,113.24,23.19,7,M,36,M32-38
2121198,2121199,-9222956879900151005,2016-05-06 20:49:39,0.00,0.00,6,M,36,M32-38
2156736,2156737,-9222956879900151005,2016-05-07 14:49:24,0.00,0.00,7,M,36,M32-38
2517594,2517595,-9222956879900151005,2016-05-07 12:09:01,113.24,23.19,7,M,36,M32-38
554350,554351,-9222956879900151005,2016-05-06 15:34:16,0.00,0.00,6,M,36,M32-38
2158245,2158246,-9222956879900151005,2016-05-07 15:39:24,0.00,0.00,7,M,36,M32-38


In [43]:
train = train.join(train['device_id'].apply(just_int), how='left')
test = test.join(test['device_id'].apply(just_int), how='left')

#train['device_hex'] = train['device_id'].apply(tohex_str)
#test['device_hex'] = test['device_id'].apply(tohex_str)

In [45]:
extracted_id_train = train['device_id'].apply(just_int)
extracted_id_test = test['device_id'].apply(just_int)

In [73]:


ff_train = train['device_hex'].apply(extract_first_four)
ff_test = test['device_hex'].apply(extract_first_four)

train = train.join(ff_train, how='left')
test = test.join(ff_test, how='left')

In [74]:
train.columns = [u'device_id', u'gender', u'age', u'group', u'device_hex', u'ff', u'mf', u'nf', u'bf']

In [75]:
test.columns = [u'device_id', u'device_hex', u'ff', u'mf', u'nf', u'bf']

In [85]:
train_test = pd.concat([train[['ff','mf','nf', 'bf','group']], test[['ff','mf','nf','bf']]], axis=0)

In [31]:
def count_number(val):
    temp = np.zeros(16)
    for i in val:
        idx = int(i, 16)
        temp[idx] = temp[idx] + 1
    return pd.Series(temp)

extracted_id_train = train['device_hex'].apply(count_number)
extracted_id_test = test['device_hex'].apply(count_number)

In [5]:
def extract_to_array(val):
    result = []
    bin_array = [int(x) for x in bin(int(val, 16))[2:].zfill(64)]
    return pd.Series(bin_array)

extracted_id_train = train['device_hex'].apply(extract_to_array)
extracted_id_test = test['device_hex'].apply(extract_to_array)

In [17]:
from scipy.sparse import csr_matrix
sparse_device_id_train = csr_matrix(extracted_id_train)
sparse_device_id_test = csr_matrix(extracted_id_test)

In [18]:
def save_sparse(filename, xmtr):
    np.savez(filename,data = xmtr.data ,indices= xmtr.indices,
             indptr =xmtr.indptr, shape=xmtr.shape )
    
def load_sparse(filename):
    tmp = np.load(filename)
    return csr_matrix((tmp['data'], tmp['indices'], tmp['indptr']), shape= tmp['shape'])

In [20]:
Xtrain = load_sparse('WorkSpace/sprm_train_v2.npz')
Xtest = load_sparse('WorkSpace/sprm_test_v2.npz')

In [21]:
from scipy.sparse import hstack

Xtrain2 = hstack((sparse_device_id_train, Xtrain), format='csr')
Xtest2 =  hstack((sparse_device_id_test, Xtest), format='csr')

In [26]:
save_sparse('WorkSpace/sprm_train_v3', Xtrain2)
save_sparse('WorkSpace/sprm_test_v3', Xtest2)

In [21]:
train = train.join(extracted_id_train, how='left')

In [22]:
train.head(20)

,device_id,gender,age,group,device_hex,more_than_30,0,1,2,3,...,54,55,56,57,58,59,60,61,62,63
0,-8076087639492063270,M,35,M32-38,8febf914b1270fda,1,1,0,0,0,...,1,1,1,1,0,1,1,0,1,0
1,-2897161552818060146,M,35,M32-38,d7cb36eae35c448e,1,1,1,0,1,...,0,0,1,0,0,0,1,1,1,0
2,-8260683887967679142,M,35,M32-38,8d5c27c557e4cd5a,1,1,0,0,0,...,0,1,0,1,0,1,1,0,1,0
3,-4938849341048082022,M,30,M29-31,bb75aeb103a8319a,0,1,0,1,1,...,0,1,1,0,0,1,1,0,1,0
4,245133531816851882,M,30,M29-31,0366e3a5fe7af9aa,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
5,-1297074871525174196,F,24,F24-26,edffdd46cba8144c,0,1,1,1,0,...,0,0,0,1,0,0,1,1,0,0
6,236877999787307864,M,36,M32-38,03498f49286d7358,1,0,0,0,0,...,1,1,0,1,0,1,1,0,0,0
7,-8098239495777311881,M,38,M32-38,8f9d4615bc27a377,1,1,0,0,0,...,1,1,0,1,1,1,0,1,1,1
8,176515041953473526,M,33,M32-38,02731b7ed52663f6,1,0,0,0,0,...,1,1,1,1,1,1,0,1,1,0
9,1596610250680140042,F,36,F33-42,16284c905489810a,1,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0


In [52]:
y_encoder = LabelEncoder().fit(train['gender'])
y = y_encoder.transform(train['gender'])
yclasses_ = len(y_encoder.classes_)
print 'total number of target classes: ', yclasses_, y_encoder.classes_

total number of target classes:  2 ['F' 'M']


In [7]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression, LogisticRegression

In [68]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold, StratifiedKFold
skf = KFold(n=len(train), n_folds= 10, random_state= 42)
clf = LinearRegression(normalize=True)
results = cross_val_score(clf, extracted_id_train.as_matrix(), train['age'], cv=skf, verbose=1)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -0.02 (0.02) MSE


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.1s finished


In [67]:
extracted_id_train.as_matrix()

array([[1, 8, 0, ..., 2, 7, 0],
       [1, 2, 8, ..., 1, 4, 6],
       [1, 8, 2, ..., 1, 4, 2],
       ..., 
       [0, 0, 1, ..., 3, 1, 2],
       [0, 1, 3, ..., 8, 5, 1],
       [0, 0, 0, ..., 3, 4, 7]])

In [66]:
clf.fit(extracted_id_train.as_matrix(), y)
j = clf.predict(extracted_id_train.as_matrix())
#from sklearn.metrics import log_loss
#print log_loss(y, j)
j

array([ 0.64350579,  0.64454208,  0.64694651, ...,  0.64688255,
        0.63058201,  0.65117658])

In [32]:
y = train['age']
X = extracted_id_train.as_matrix()
from keras.models import Sequential
from keras.layers import Dense, Activation
def build_model():
    model = Sequential()
    model.add(Dense(32, input_dim=64))
    model.add(Dense(16, input_dim=64))
    model.add(Dense(16, input_dim=64))
    model.add(Dense(1))
    model.add(Activation("linear"))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [34]:
from keras.wrappers.scikit_learn import KerasRegressor
estimator = KerasRegressor(build_fn=build_model, nb_epoch=40, batch_size=15, verbose=1)

In [35]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

kfold = KFold(n=len(X), n_folds= 5, random_state= 42)
results = cross_val_score(estimator, X, y, cv=kfold, verbose=1)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/40
59716/59716 [==============================] - 2s - loss: 105.4491     
Epoch 2/40
59716/59716 [==============================] - 2s - loss: 90.7550     
Epoch 3/40
59716/59716 [==============================] - 2s - loss: 89.5218     
Epoch 4/40
59716/59716 [==============================] - 2s - loss: 89.3329     
Epoch 5/40
59716/59716 [==============================] - 2s - loss: 89.1289     
Epoch 6/40
59716/59716 [==============================] - 2s - loss: 88.9975     
Epoch 7/40
59716/59716 [==============================] - 2s - loss: 88.7660     
Epoch 8/40
59716/59716 [==============================] - 2s - loss: 88.6081     
Epoch 9/40
59716/59716 [==============================] - 2s - loss: 88.4405     
Epoch 10/40
59716/59716 [==============================] - 2s - loss: 88.3689     
Epoch 11/40
59716/59716 [==============================] - 2s - loss: 88.1339     
Epoch 12/40
59716/59716 [==============================] - 2s - loss: 88.0844     
Epoch 13/40


KeyboardInterrupt: 

In [38]:
y_predict = clf.predict(X)
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predict)

array([[26722,    19],
       [    8, 47896]])

In [159]:
proba_test = clf.predict_proba(extracted_id_test.as_matrix())
proba_predict = clf.predict(extracted_id_test.as_matrix())

In [166]:
check = pd.DataFrame(proba_test, columns=['Female', 'Male'])

In [168]:
np.save('WorkSpace/train_predict_v1', predict)
np.save('WorkSpace/test_predict_v1', proba_test)

In [135]:
train.head(10)

,device_id,gender,age,group,device_hex,0,1,2,3,4,...,8,9,10,11,12,13,14,15,Female,Male
0,-8076087639492063270,M,35,M32-38,8febf914b1270fda,8,15,14,11,15,...,11,1,2,7,0,15,13,10,0.14,0.86
1,-2897161552818060146,M,35,M32-38,d7cb36eae35c448e,13,7,12,11,3,...,14,3,5,12,4,4,8,14,0.16,0.84
2,-8260683887967679142,M,35,M32-38,8d5c27c557e4cd5a,8,13,5,12,2,...,5,7,14,4,12,13,5,10,0.18,0.82
3,-4938849341048082022,M,30,M29-31,bb75aeb103a8319a,11,11,7,5,10,...,0,3,10,8,3,1,9,10,0.18,0.82
4,245133531816851882,M,30,M29-31,366e3a5fe7af9aa,0,3,6,6,14,...,15,14,7,10,15,9,10,10,0.20,0.80
5,-1297074871525174196,F,24,F24-26,edffdd46cba8144c,14,13,15,15,13,...,12,11,10,8,1,4,4,12,0.74,0.26
6,236877999787307864,M,36,M32-38,3498f49286d7358,0,3,4,9,8,...,2,8,6,13,7,3,5,8,0.18,0.82
7,-8098239495777311881,M,38,M32-38,8f9d4615bc27a377,8,15,9,13,4,...,11,12,2,7,10,3,7,7,0.06,0.94
8,176515041953473526,M,33,M32-38,2731b7ed52663f6,0,2,7,3,1,...,13,5,2,6,6,3,15,6,0.18,0.82
9,1596610250680140042,F,36,F33-42,16284c905489810a,1,6,2,8,4,...,5,4,8,9,8,1,0,10,0.72,0.28


In [39]:
skf = StratifiedKFold(y, 5, shuffle=True)

In [40]:
from sklearn.cross_validation import train_test_split 
Xt, Xv, yt, yv = train_test_split(X,y, train_size=.9)

In [41]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 10
param['silent'] = 1
param['num_class'] = 2
param['eval_metric'] = 'mlogloss'
d_train = xgb.DMatrix(Xt, label=yt)
d_valid = xgb.DMatrix(Xv, label=yv)
    
watchlist = [(d_train,'train'), (d_valid, 'eval')]
num_round = 5000
bst = xgb.train(param, d_train, num_round, watchlist, early_stopping_rounds=50, verbose_eval=20 );
scores_val = bst.predict(d_valid)
print('final logloss val {}'.format(log_loss(yv, scores_val))) 

[0]	train-mlogloss:0.692164	eval-mlogloss:0.69236
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 50 rounds.
[20]	train-mlogloss:0.676509	eval-mlogloss:0.679593
[40]	train-mlogloss:0.666023	eval-mlogloss:0.671046
[60]	train-mlogloss:0.658584	eval-mlogloss:0.665307
[80]	train-mlogloss:0.654079	eval-mlogloss:0.66148
[100]	train-mlogloss:0.651007	eval-mlogloss:0.658912
[120]	train-mlogloss:0.648839	eval-mlogloss:0.65719
[140]	train-mlogloss:0.647272	eval-mlogloss:0.656044
[160]	train-mlogloss:0.645951	eval-mlogloss:0.655273
[180]	train-mlogloss:0.644873	eval-mlogloss:0.65476
[200]	train-mlogloss:0.643516	eval-mlogloss:0.654422
[220]	train-mlogloss:0.642294	eval-mlogloss:0.654236
[240]	train-mlogloss:0.640715	eval-mlogloss:0.654152
[260]	train-mlogloss:0.639265	eval-mlogloss:0.65418
[280]	train-mlogloss:0.638111	eval-mlogloss:0.654187
Stopping. Best iteration:
[249]	train-mlogloss:0.640075	eval-mlog

In [187]:
whole_train = xgb.DMatrix(X)
whole_test = xgb.DMatrix(extracted_id_test.as_matrix())
whole_predict_train = bst.predict(whole_train)
whole_predict_test = bst.predict(whole_test)

In [184]:
train = train.drop(['Female','Male'], axis=1)
train = train.join(pd.DataFrame(whole_predict, columns=['Female', 'Male']))

In [189]:
test_pd = pd.DataFrame(whole_predict_test, columns=['Female', 'Male'])

In [191]:
np.save('WorkSpace/train_predict_gender_xgb', whole_predict_train)
np.save('WorkSpace/test_predict_gender_xgb', whole_predict_test)